In [0]:
from pyspark.sql.functions import *
customer=spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load('/Volumes/sandeshmsdatabricks/bronze/bronzevol/dim_customers.csv')



In [0]:
sales=spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load('/Volumes/sandeshmsdatabricks/bronze/bronzevol/fact_sales.csv')


### Question
Create a DataFrame that shows all sales with customer details by joining sales to customers on customer_id using an inner join. The output should have:

sales_id, date, customer_id, name, email, location, product_id, quantity, discount, total_amount.

Then show the first 3 rows.

In [0]:
inner_join=sales.join(customer,on="customer_id",how="inner").select("sales_id","date","customer_id","name","email","location","product_id","quantity","discount","total_amount").show(3)


+--------+----------+-----------+--------------------+--------------------+---------------+----------+--------+--------+------------+
|sales_id|      date|customer_id|                name|               email|       location|product_id|quantity|discount|total_amount|
+--------+----------+-----------+--------------------+--------------------+---------------+----------+--------+--------+------------+
|       1|2024-09-21|         63|       Chad Gill DDS|amanda87@gibson-e...|       Hallstad|        38|       3|   29.24|      113.25|
|       2|2025-03-02|         96|   Samantha Robinson|justin67@hudson.info|     Carriebury|        97|       4|    1.57|     1038.44|
|       3|2024-09-22|         52|Dr. Charles Monto...| zstevens@harvey.com|Washingtonmouth|        23|       4|   20.47|      475.94|
+--------+----------+-----------+--------------------+--------------------+---------------+----------+--------+--------+------------+
only showing top 3 rows


### Left Join with slect and filtering

In [0]:
sales.join(customer,on="customer_id",how="left").select("sales_id","date","customer_id","name","email","location","product_id","quantity","discount","total_amount").filter(col("name").isNull() ).show()

+--------+----+-----------+----+-----+--------+----------+--------+--------+------------+
|sales_id|date|customer_id|name|email|location|product_id|quantity|discount|total_amount|
+--------+----+-----------+----+-----+--------+----------+--------+--------+------------+
+--------+----+-----------+----+-----+--------+----------+--------+--------+------------+



### Outer JOin 

In [0]:
sales.join(customer,on="customer_id",how="outer").select("sales_id","date","customer_id","name","email","location","product_id","quantity","discount","total_amount").show(2)

+--------+----------+-----------+------------------+--------------------+-----------+----------+--------+--------+------------+
|sales_id|      date|customer_id|              name|               email|   location|product_id|quantity|discount|total_amount|
+--------+----------+-----------+------------------+--------------------+-----------+----------+--------+--------+------------+
|     313|2025-01-03|         94|Christopher Phelps|phernandez@gmail.com|Port Amanda|       100|       4|   16.77|      194.46|
|     369|2024-10-25|         94|Christopher Phelps|phernandez@gmail.com|Port Amanda|        83|       3|    0.34|      504.65|
+--------+----------+-----------+------------------+--------------------+-----------+----------+--------+--------+------------+
only showing top 2 rows


### Right Join

In [0]:
sales.join(customer, on="customer_id", how="right")\
     .select("sales_id", "date", "customer_id", "name", "email", "location",
             "product_id", "quantity", "discount", "total_amount")\
     .show(3)


+--------+----------+-----------+------------------+--------------------+------------+----------+--------+--------+------------+
|sales_id|      date|customer_id|              name|               email|    location|product_id|quantity|discount|total_amount|
+--------+----------+-----------+------------------+--------------------+------------+----------+--------+--------+------------+
|     968|2025-04-10|          1|     Madison Smith|williamsteresa@ca...|  New Darryl|        92|       4|   28.46|     1024.62|
|     395|2025-05-30|          2|        Kyle Brown|schroedermarco@ho...|Lake Anthony|        19|       2|    2.76|      425.54|
|     930|2024-10-18|          3|Alexander Gonzalez|  fgriffin@yahoo.com| Port Joseph|         8|       1|   22.25|      337.25|
+--------+----------+-----------+------------------+--------------------+------------+----------+--------+--------+------------+
only showing top 3 rows


### Broadcast Join
- It copies smallest dataframe to all worker node,avoiding large shuffle operation.
- Ideal when we have one large dataframe and one small dataframe

In [0]:
sales.join(broadcast(customer), on="customer_id", how="right")\
     .select("sales_id", "date", "customer_id", "name", "email", "location",
             "product_id", "quantity", "discount", "total_amount")\
     .show(3)

+--------+----------+-----------+------------------+--------------------+------------+----------+--------+--------+------------+
|sales_id|      date|customer_id|              name|               email|    location|product_id|quantity|discount|total_amount|
+--------+----------+-----------+------------------+--------------------+------------+----------+--------+--------+------------+
|     968|2025-04-10|          1|     Madison Smith|williamsteresa@ca...|  New Darryl|        92|       4|   28.46|     1024.62|
|     395|2025-05-30|          2|        Kyle Brown|schroedermarco@ho...|Lake Anthony|        19|       2|    2.76|      425.54|
|     930|2024-10-18|          3|Alexander Gonzalez|  fgriffin@yahoo.com| Port Joseph|         8|       1|   22.25|      337.25|
+--------+----------+-----------+------------------+--------------------+------------+----------+--------+--------+------------+
only showing top 3 rows


### Question
Write PySpark code to:

Join sales and customer on customer_id with an inner join.

Define a window partitioned by customer_id and ordered by date.

Add columns purchase_rank (using rank()) and running_total (using sum() over the window).

Select customer_id, date, purchase_rank, running_total, and show 10 rows.

In [0]:
from pyspark.sql.window import *
from pyspark.sql.functions import *


customer_sales=sales.join(customer,
                          on="customer_id",
                          how="inner")

windowspec=Window.partitionBy("customer_id").orderBy("date")

customer_sales=customer_sales.withColumn("purchase_rank",rank().over(windowspec))\
    .select("customer_id","date","purchase_rank").orderBy("date")\
    .show(10)

+-----------+----------+-------------+
|customer_id|      date|purchase_rank|
+-----------+----------+-------------+
|          1|2024-06-28|            1|
|        164|2024-06-30|            1|
|        141|2024-07-01|            1|
|        160|2024-07-01|            1|
|         85|2024-07-03|            1|
|        165|2024-07-03|            1|
|         19|2024-07-03|            1|
|        180|2024-07-04|            1|
|         75|2024-07-04|            1|
|        173|2024-07-04|            1|
+-----------+----------+-------------+
only showing top 10 rows


### Union
- .union() is to combine two dataframe with the identical schemas.
- .unionByName() is used if columns have same name but ordered diffrently

In [0]:
subset1 = sales.limit(5)
subset2 = sales.subtract(subset1)  

union_df = subset1.union(subset2)
union_df.show(10)

+--------+-----------+----------+--------+--------+--------+----------+------------+
|sales_id|customer_id|product_id|store_id|quantity|discount|      date|total_amount|
+--------+-----------+----------+--------+--------+--------+----------+------------+
|       1|         63|        38|      12|       3|   29.24|2024-09-21|      113.25|
|       2|         96|        97|      14|       4|    1.57|2025-03-02|     1038.44|
|       3|         52|        23|      14|       4|   20.47|2024-09-22|      475.94|
|       4|         96|        63|       2|       4|   14.04|2024-08-22|     1427.73|
|       5|        132|        15|       5|       3|   17.38|2024-11-11|      235.47|
|     397|        160|        59|      10|       3|   19.25|2024-09-10|       66.35|
|     808|         47|        53|      11|       1|   27.31|2025-02-21|      341.68|
|     538|         36|        62|       8|       4|   13.86|2024-10-05|      480.04|
|     882|         95|        46|      15|       2|    4.68|2025-